In [26]:
sc.stop()

from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from operator import add
conf = SparkConf().setAppName("MLlib")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
# sc.stop() if need to 

spark = SparkSession(sc)

In [27]:
#Q1
def parseLine(x):
    pay_id = x.split(',')[1].replace('"','')
    amt = float(x.split(',')[4].replace('"',''))
    return (pay_id, amt)


pay = sc.textFile("payment.csv")
pay_first = pay.first()
pay.filter(lambda x: x != pay_first).map(parseLine).reduceByKey(add)\
.sortBy(lambda x: x[1], ascending = False)\
.toDF()\
.take(5)

[Row(_1='526', _2=221.5500000000001),
 Row(_1='148', _2=216.5400000000001),
 Row(_1='144', _2=195.58000000000007),
 Row(_1='137', _2=194.61000000000007),
 Row(_1='178', _2=194.61000000000007)]

In [28]:
#Q2
def parseLine2(x):
    rental_id = x.split(',')[0].replace('"','')
    rental_date = x.split(',')[1].replace('"','')
    return_date = x.split(',')[4].replace('"','')
    staff_id = x.split(',')[5].replace('"','')
    return (rental_id, rental_date, return_date, staff_id)

rental = sc.textFile("rental.csv")
rental_first = rental.first()
rental.filter(lambda x: x != rental_first).map(parseLine2)\
.sortBy(lambda x: x[2], ascending = True)\
.toDF()\
.take(5)

[Row(_1='32', _2='2005-05-25 04:06:21', _3='2005-05-25 23:55:21', _4='1'),
 Row(_1='21', _2='2005-05-25 01:59:46', _3='2005-05-26 01:01:46', _4='2'),
 Row(_1='14', _2='2005-05-25 00:31:15', _3='2005-05-26 02:56:15', _4='1'),
 Row(_1='16', _2='2005-05-25 00:43:11', _3='2005-05-26 04:42:11', _4='2'),
 Row(_1='22', _2='2005-05-25 02:19:23', _3='2005-05-26 04:52:23', _4='2')]

In [29]:
#Q3
staff = rental.filter(lambda x: x != rental_first).map(parseLine2)\
        .sortBy(lambda x: x[3], ascending = True)\
        .map(lambda x: (x[3]))\
        .countByKey()
staff

defaultdict(int, {'1': 8040, '2': 8004})

In [30]:
#Q4
staff['Staff_id 1'] = "Mike","Hillyer", staff['1']
staff['Staff_id 2'] = "Jon","Stephens", staff['2']
staff

defaultdict(int,
            {'1': 8040,
             '2': 8004,
             'Staff_id 1': ('Mike', 'Hillyer', 8040),
             'Staff_id 2': ('Jon', 'Stephens', 8004)})

In [31]:
#Q6

from datetime import datetime
from operator import add

def parseLine1(x):
    pay_id = x.split(',')[0].replace('"','')
    cust_id = x.split(',')[1].replace('"','')
    pay_amt = float(x.split(',')[4].replace('"',''))
    pay_date = x.split(',')[5].replace('"','').split('-')[0]
    return (cust_id, pay_amt, pay_date)

pay.filter(lambda x: x != pay_first).map(parseLine1)\
.filter(lambda x: (x[2] == '2005'))\
.sortBy(lambda x: x[1], ascending = False)\
.take(5)


[('13', 11.99, '2005'),
 ('116', 11.99, '2005'),
 ('195', 11.99, '2005'),
 ('196', 11.99, '2005'),
 ('204', 11.99, '2005')]